# Some Q's

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as sm
#from sklearn import tree
import pydotplus
import graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score,mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelBinarizer
from pathlib import Path
from DataCleanUp import *

from model import modelit

M = modelit()
M.predict(ret=False)
print("Score from Random Forest = ",M.score_v)
#M.plotYvsPredit() # Does not work in Jupyter. so c below

In [ ]:
fig, ax = plt.subplots()
ax.scatter(M.yP, M.pred, edgecolors=(0, 0, 0))
ax.plot([M.yP.min(), M.yP.max()], [M.yP.min(), M.yP.max()], 'k--', lw=4)
ax.set_xlabel('Observed')
ax.set_ylabel('Predicted')


## What's the most important feature?

In [ ]:
print(M.df_train.columns)
result = sm.ols(formula="USD_hamP ~ USD_Hest ", data=M.df_train).fit()
print(result.summary2())

#### as expected estimate (high) just dominates the field with Adj R2 of 0.928 (even I include NaNs Adj R2 is still above 0.7), but including this term will introduce multicollinearity prb with others, as this is pretty much what an experts opinion after all data is considered. Plus, not all of work has this number available. Definitely model with it, but domination should b considered with a grain of salt.

In [ ]:
result = sm.ols(formula="USD_DjiP ~ artist_name + auct_yr + measure_hwd_cm + unique +artist_birth_year + exec_pmortem + category", data=M.df_train.fillna(-999)).fit()
print(result.summary2())

#### hammer_price is not a good item to try to predict. We have data going back to 1985. Asset price has risen considerably since. So, if keep on using the USD equivalent, then we will miss all signals. I have imported DJI for this reason to find relative price to stock market. With estimate P, it's dominance & equivalent pricing given the date, would make any model seem like a good model, but remove that its unlikely any model will be able to go beyond asset inflation bias

In [ ]:
M.df_train.category.value_counts()

In [ ]:
M = modelit()
M.predict(ret=False, colnms='measure_hwd_cm,category')
print("Score from Random Forest = ",M.score_v)

#### artist name + measurement + category didn't make interesting model either linearly or with Random Forest

## Creating uncorrleated Return

### I usually do this by few steps.
#### This is standard Modus Operandi of any Prop Trading desk or Hedge Fund
1st create returns based on few features.
Category
Artist
Genre
country
etc...
#### create return series & look for correlation coeff & more opposite or close to 0 as possible the better.
### Use an optimizer (I always used commercial optimizer so could not do this in Python as this requires higher math than simplex) to create bounds, such as no Genre, country, or artist may take more than X% (X=5~14%) of portfolio & consider rebalance whenever the threshold is reached.

## Recommendation category really depends on the correlation matrix one has

In [ ]:
da = transform_data(pd.read_csv('data.csv', encoding='latin-1'))
colnm = 'USD_hamP,USD_DjiP,auct_yr,measure_hwd_cm,unique,exec_pmortem,category'.split(',')
sol = da.query("artist_name =='Sol LeWitt'")[colnm].sort_values(by=['category', 'auct_yr'])
sol = sol[sol.auct_yr > 2000]
g = sol.groupby(['category','auct_yr'])

In [ ]:
sol_CmeanCnt = sol[sol.auct_yr > 2000].groupby(['category','auct_yr']).USD_DjiP.agg(['mean', 'count'])
plt.plot(sol_AmeanCnt)

### according to above price never did recover

## As for confidence intervals,
Not sure why forestci is not working. Likely overflow = Data issue due to NaN's or other large data issues
Will need to investigate further.

In [ ]:
"""/opt/miniconda3/envs/SPk/lib/python3.6/site-packages/forestci/calibration.py:86: RuntimeWarning: overflow encountered in exp
  g_eta_raw = np.exp(np.dot(XX, eta)) * mask
/opt/miniconda3/envs/SPk/lib/python3.6/site-packages/forestci/calibration.py:101: RuntimeWarning: overflow encountered in exp
  g_eta_raw = np.exp(np.dot(XX, eta_hat)) * mask
/opt/miniconda3/envs/SPk/lib/python3.6/site-packages/forestci/calibration.py:102: RuntimeWarning: invalid value encountered in true_divide
  g_eta_main = g_eta_raw / sum(g_eta_raw)
  """

### The idea here is that as with Ensemble models, number of tree (30 in this case) predicting each Y. So, one can make variance & mean based on this, which will be much more accurate than comparing simple variance of entire Y predicted vs actual.
Unfortunately, the code I am using is failing due to overflow, which means I will have to debug line by line or just create my own. Not 2~3 hr job.